# extracting hemodynamic state equation values

from Jan's ODE model

In [1]:
#~ lb.variables_of_interest = ["V", "W", "Z"]

In [ ]:
%matplotlib inline

%load_ext autoreload

%autoreload 2

import numpy as np
import matplotlib.pylab as plt

from tvb.simulator.lab import *

from tvb.simulator.backend.nb_mpr import NbMPRBackend

In [ ]:
N = 2
conn = connectivity.Connectivity()
conn.motif_all_to_all(number_of_regions=N)
conn.centres_spherical(number_of_regions=N)
conn.speed = np.r_[np.inf]

conn.configure()

weighting = patterns.StimuliRegion.get_default_weights(N)
weighting[0] = 4.

# temporal profile
eqn_t = equations.Gaussian()
eqn_t.parameters["midpoint"] = 11000.0
eqn_t.parameters["sigma"] = 100.0

stimulus = patterns.StimuliRegion(temporal=eqn_t,
                                  connectivity=conn,
                                  weight=weighting)

sim = simulator.Simulator(
    model = models.MontbrioPazoRoxin(),
    connectivity = conn,
    integrator = integrators.HeunStochastic(
        dt = 0.01, 
        noise = noise.Additive(nsig=np.r_[0.01,0.02])
    ),
    initial_conditions=np.zeros( (1,2,N,1) ),
    stimulus=stimulus,
    monitors=[monitors.Raw()]
).configure()

backend = NbMPRBackend()

%%capture
(raw_t, raw_d), = backend.run_sim(sim, simulation_length=30000)

raw_t.shape, raw_d.shape

plt.plot(raw_t, raw_d[:, 0,:,0])

In [ ]:
def replay_monitor(raw_d, sim):
    """ Assumes that the raw_d was produced in consistency with the simulator used."""
    assert len(sim.monitors) == 1, "Only one monitor at a time can be replayed."    
    res = [out for out in [ sim.monitors[0].record(step, sim.model.observe(state)) for step, state in enumerate(raw_d)] if out is not None]
    b_t, b_d = zip(*res)
    b_t, b_d = np.array(b_t), np.array(b_d)
    return b_t, b_d

sim = simulator.Simulator(
    model = models.MontbrioPazoRoxin(variables_of_interest=['r']),
    connectivity = conn,
    integrator = integrators.HeunStochastic(
        dt = 0.01, 
        noise = noise.Additive(nsig=np.r_[0.01,0.02])
    ),
    initial_conditions=np.zeros( (1,2,N,1) ),
    stimulus=stimulus,
    monitors=[monitors.Bold(period=10., include_svars=True)], # period=500 #monitors.BoldBalloonWindkessel(period=10., V0=0.02, tau_s=0.65, alpha=0.32, include_svars=True)],

).configure()

b_t, b_d = replay_monitor(raw_d, sim)

fig, axs = plt.subplots(nrows=6, sharex=True, figsize=(8,10))
axs[0].plot(raw_t,raw_d[:,0,0,0])
axs[0].set(ylabel='r')
for i, (ax, label) in enumerate( zip(axs[1:], ['BOLD', 's', 'f', 'v', 'q'])):
    ax.plot(b_t[:], b_d[:,i,0,0])
    ax.set(ylabel=label)

In [ ]:
conn = connectivity.Connectivity.from_file()

# configure stimulus spatial pattern
weighting = numpy.zeros((76, ))
weighting[[0, 7, 13, 33, 42]] = numpy.array([2.0 ** -2, 2.0 ** -3, 2.0 ** -4, 2.0 ** -5, 2.0 ** -6])

# temporal profile
eqn_t = equations.Gaussian()
eqn_t.parameters["midpoint"] = 25000.0
eqn_t.parameters["sigma"] = 1.0 #2000.0

stimulus = patterns.StimuliRegion(temporal=eqn_t,
                                  connectivity=conn,
                                  weight=weighting)

sim = simulator.Simulator(
    model=models.Generic2dOscillator(a=numpy.array([0.5])),
    connectivity=conn,
    coupling=coupling.Linear(a=numpy.array([0.0126])),
    integrator=integrators.HeunDeterministic(dt=0.5),
    monitors=(
        monitors.TemporalAverage(period=1.0),
        monitors.Bold(period=500),
        monitors.ProgressLogger(period=5e3),
        ),
    stimulus=stimulus,
    simulation_length=60e3, # 1 minute simulation
).configure()

(tavg_time, tavg_data), (bold_time, bold_data), _ = sim.run()

#Make the lists numpy.arrays for easier use.
LOG.info("Converting result to array...")
TAVG_TIME = numpy.array(tavg_time)
BOLD_TIME = numpy.array(bold_time)
BOLD = numpy.array(bold_data)
TAVG = numpy.array(tavg_data)

#Create TimeSeries instance
tsr = TimeSeriesRegion(connectivity = conn,  # inc'd this
                       data = TAVG,
                       time = TAVG_TIME,
                       sample_period = 2.)
tsr.configure()

#Create and run the monitor/analyser
bold_model = bold.BalloonModel(time_series = tsr)
bold_data  = bold_model.evaluate()

plt.figure(figsize=(12,8))

#plt.subplot(212)
plt.plot(bold_data.time[25000:45000], bold_data.data[25000:45000, 0, [42], 0])
plt.ylabel("BOLD")

plt.xlabel('Time (ms)')